Notebook to make callender df for DataSet3

In [1]:
import pandas as pd
import numpy as np

#### get data for target

In [2]:
df = pd.read_pickle('./../../data/interim/D3_dailyTS/EDdaily_occupancy_classRollQ90_clipped.pkl')
df.head()

,EDMeanOcc,flag_target
dt_date,,
2011-09-01,28.000000,0
2011-09-02,36.833333,0
2011-09-03,29.291667,0
2011-09-04,36.833333,0
2011-09-05,32.916667,0


In [3]:
df = df.reset_index()

In [4]:
df.head()

,dt_date,EDMeanOcc,flag_target
0,2011-09-01,28.000000,0
1,2011-09-02,36.833333,0
2,2011-09-03,29.291667,0
3,2011-09-04,36.833333,0
4,2011-09-05,32.916667,0


In [5]:
df['year'] = df['dt_date'].apply(lambda x: x.year)

In [6]:
df['month'] = df['dt_date'].apply(lambda x: x.month)

In [7]:
df['dayofweek'] = df['dt_date'].apply(lambda x: x.day_name()) #.day

Add holiday feature

In [8]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, GoodFriday, EasterMonday, next_monday, DateOffset, MO,next_monday_or_tuesday

Based on TERRIBLE blog here: https://www.tobiolabode.com/blog-1/2019/1/1/pandas-for-uk-hoildays,  https://medium.com/@Tobi_olabode/pandas-for-uk-hoildays-f561558e854f

With implementation here: https://stackoverflow.com/questions/29688899/pandas-checking-if-a-date-is-a-holiday-and-assigning-boolean-value

In [9]:
class EnglandAndWalesHolidayCalendar(AbstractHolidayCalendar):
 rules = [
 Holiday("New Years Day", month=1, day=1, observance=next_monday),
 GoodFriday,
 EasterMonday,
 Holiday("Early May bank holiday",
 month=5, day=1, offset=DateOffset(weekday=MO(1))),
 Holiday("Spring bank holiday",
 month=5, day=31, offset=DateOffset(weekday=MO(-1))),
 Holiday("Summer bank holiday",
 month=8, day=31, offset=DateOffset(weekday=MO(-1))),
 Holiday("Christmas Day", month=12, day=25, observance=next_monday),
 Holiday("Boxing Day",
 month=12, day=26, observance=next_monday_or_tuesday)
 ]

In [10]:
# from pandas.tseries.holiday import EnglandAndWalesHolidayCalendar as calendar

calendar = EnglandAndWalesHolidayCalendar

dr = df.dt_date.values

In [11]:
# dr = df.
# df = pd.DataFrame()
# df['Date'] = dr

cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())

# df['Holiday'] = 
df['holiday'] = df['dt_date'].isin(holidays).astype(int)
# print(df)

In [12]:
df.holiday.value_counts()

0    2531
1      56
Name: holiday, dtype: int64

In [13]:
df.head()

,dt_date,EDMeanOcc,flag_target,year,month,dayofweek,holiday
0,2011-09-01,28.000000,0,2011,9,Thursday,0
1,2011-09-02,36.833333,0,2011,9,Friday,0
2,2011-09-03,29.291667,0,2011,9,Saturday,0
3,2011-09-04,36.833333,0,2011,9,Sunday,0
4,2011-09-05,32.916667,0,2011,9,Monday,0


Add December decanting flag feature

In [14]:
df['christmas_period'] = 0

In [15]:
df['day'] = df['dt_date'].apply(lambda x: x.day)

In [16]:
df.head()

,dt_date,EDMeanOcc,flag_target,year,month,dayofweek,holiday,christmas_period,day
0,2011-09-01,28.000000,0,2011,9,Thursday,0,0,1
1,2011-09-02,36.833333,0,2011,9,Friday,0,0,2
2,2011-09-03,29.291667,0,2011,9,Saturday,0,0,3
3,2011-09-04,36.833333,0,2011,9,Sunday,0,0,4
4,2011-09-05,32.916667,0,2011,9,Monday,0,0,5


In [17]:
dec_dates = [23,24,25,26,27,28,29,30,31]
df.loc[(df.day.isin(dec_dates)) & (df.month == 12), 'christmas_period'] = 1

jan_dates = [1,2,3]
df.loc[(df.day.isin(jan_dates)) & (df.month == 1), 'christmas_period'] = 1

get cylic vars

In [18]:
df['day_count'] = np.arange(0,len(df))

In [19]:
df.head()

,dt_date,EDMeanOcc,flag_target,year,month,dayofweek,holiday,christmas_period,day,day_count
0,2011-09-01,28.000000,0,2011,9,Thursday,0,0,1,0
1,2011-09-02,36.833333,0,2011,9,Friday,0,0,2,1
2,2011-09-03,29.291667,0,2011,9,Saturday,0,0,3,2
3,2011-09-04,36.833333,0,2011,9,Sunday,0,0,4,3
4,2011-09-05,32.916667,0,2011,9,Monday,0,0,5,4


In [20]:

# days_in_cycle = 7
# df['week_sin'] = np.sin(2*np.pi*df.day_count/days_in_cycle)
# df['week_cos'] = np.cos(2*np.pi*df.day_count/days_in_cycle)
days_in_cycle = 365
df['year_sin'] = np.sin(2*np.pi*df.day_count/days_in_cycle)
df['year_cos'] = np.cos(2*np.pi*df.day_count/days_in_cycle)

df.drop('day_count', axis=1, inplace=True)

df.head()

,dt_date,EDMeanOcc,flag_target,year,month,dayofweek,holiday,christmas_period,day,year_sin,year_cos
0,2011-09-01,28.000000,0,2011,9,Thursday,0,0,1,0.000000,1.000000
1,2011-09-02,36.833333,0,2011,9,Friday,0,0,2,0.017213,0.999852
2,2011-09-03,29.291667,0,2011,9,Saturday,0,0,3,0.034422,0.999407
3,2011-09-04,36.833333,0,2011,9,Sunday,0,0,4,0.051620,0.998667
4,2011-09-05,32.916667,0,2011,9,Monday,0,0,5,0.068802,0.997630


OneHotEnvode the weekdays

In [21]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [22]:
ohe = OneHotEncoder()

feature_arr = ohe.fit_transform(df[['dayofweek']]).toarray()

In [23]:
feature_labels = ohe.categories_
feature_labels = np.array(feature_labels).ravel()

days = pd.DataFrame(feature_arr, columns=feature_labels)

In [24]:
df = df.merge(days, right_index=True,left_index=True)

In [25]:
df = df.set_index('dt_date').drop(['EDMeanOcc','flag_target','day','dayofweek'],axis=1)

In [26]:
df.index.name = ''
df.head()

,year,month,holiday,christmas_period,year_sin,year_cos,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
,,,,,,,,,,,,,
2011-09-01,2011,9,0,0,0.000000,1.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2011-09-02,2011,9,0,0,0.017213,0.999852,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-09-03,2011,9,0,0,0.034422,0.999407,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2011-09-04,2011,9,0,0,0.051620,0.998667,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2011-09-05,2011,9,0,0,0.068802,0.997630,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df.to_pickle('./../../data/interim/D3_dailyTS/callender_weekencoded.pkl')